# 文本嵌入和向量化
# Text Embeddings and Vectorization

这个笔记本演示如何将文本转换为向量，包括TF-IDF、Word2Vec和Sentence Transformers等方法。

In [ ]:
# 导入必要的库
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from src.text_vectorizer import TextVectorizer
from src.utils import load_json

## 1. 加载示例文档

In [ ]:
# 加载示例文档
documents = load_json('../data/sample_documents.json')
texts = [doc['content'] for doc in documents]

print(f"加载了 {len(texts)} 个文档")
print("\n前3个文档内容:")
for i, text in enumerate(texts[:3]):
    print(f"文档{i+1}: {text[:100]}...")

## 2. TF-IDF 向量化

In [ ]:
# 创建TF-IDF向量化器
tfidf_vectorizer = TextVectorizer(method='tfidf')
tfidf_vectors = tfidf_vectorizer.fit_transform(texts)

print(f"TF-IDF向量形状: {tfidf_vectors.shape}")
print(f"词汇表大小: {len(tfidf_vectorizer.vectorizer.get_feature_names_out())}")

# 显示特征词
feature_names = tfidf_vectorizer.vectorizer.get_feature_names_out()
print(f"\n前20个特征词: {feature_names[:20]}")

## 3. 语义向量化 (Sentence Transformers)

In [ ]:
# 创建语义向量化器
try:
    semantic_vectorizer = TextVectorizer(method='semantic')
    semantic_vectors = semantic_vectorizer.fit_transform(texts)
    
    print(f"语义向量形状: {semantic_vectors.shape}")
    print("语义向量维度更高，能捕获更丰富的语义信息")
except Exception as e:
    print(f"语义向量化需要安装sentence-transformers: {e}")
    semantic_vectors = None

## 4. 向量化方法比较

In [ ]:
# 比较不同向量化方法
if semantic_vectors is not None:
    methods = ['TF-IDF', 'Semantic']
    vectors = [tfidf_vectors, semantic_vectors]
else:
    methods = ['TF-IDF']
    vectors = [tfidf_vectors]

for method, vec in zip(methods, vectors):
    print(f"\n{method} 向量统计:")
    print(f"  形状: {vec.shape}")
    print(f"  稀疏度: {np.mean(vec == 0)*100:.1f}%")
    print(f"  平均值: {np.mean(vec):.4f}")
    print(f"  标准差: {np.std(vec):.4f}")

## 5. 可视化向量分布

In [ ]:
# 使用PCA降维可视化
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# PCA降维到2D
pca = PCA(n_components=2)
tfidf_2d = pca.fit_transform(tfidf_vectors.toarray() if hasattr(tfidf_vectors, 'toarray') else tfidf_vectors)

plt.figure(figsize=(12, 5))

# TF-IDF可视化
plt.subplot(1, 2, 1)
plt.scatter(tfidf_2d[:, 0], tfidf_2d[:, 1], alpha=0.7)
for i, doc in enumerate(documents):
    plt.annotate(f"Doc{i+1}", (tfidf_2d[i, 0], tfidf_2d[i, 1]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)
plt.title('TF-IDF向量分布 (PCA)')
plt.xlabel('PC1')
plt.ylabel('PC2')

# 语义向量可视化
if semantic_vectors is not None:
    semantic_2d = pca.fit_transform(semantic_vectors)
    plt.subplot(1, 2, 2)
    plt.scatter(semantic_2d[:, 0], semantic_2d[:, 1], alpha=0.7, color='red')
    for i, doc in enumerate(documents):
        plt.annotate(f"Doc{i+1}", (semantic_2d[i, 0], semantic_2d[i, 1]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    plt.title('语义向量分布 (PCA)')
    plt.xlabel('PC1')
    plt.ylabel('PC2')

plt.tight_layout()
plt.show()

## 6. 文本预处理效果分析

In [ ]:
# 比较有无预处理的效果
sample_text = "这是一个测试文档！包含各种标点符号...和数字123。"

# 原始文本
print("原始文本:", sample_text)

# 预处理后
processed = tfidf_vectorizer.preprocess_text(sample_text)
print("预处理后:", processed)

# 分析预处理步骤
print("\n预处理步骤分析:")
print("1. 转小写")
print("2. 移除标点符号")
print("3. 移除数字")
print("4. 移除停用词")
print("5. 词干提取/词形还原")

## 总结

本笔记本介绍了主要的文本向量化方法:
- TF-IDF: 基于词频的统计方法
- Semantic Embeddings: 基于深度学习的语义方法

每种方法都有其适用场景，选择时需要考虑精度、速度和资源需求。

# 文本嵌入与向量化技术

本notebook将深入探讨文本向量化的各种技术，包括传统的TF-IDF方法和现代的神经网络嵌入方法。

## 学习目标
- 理解不同文本向量化方法的原理
- 学习TF-IDF、Word2Vec、Sentence Transformers的实现
- 比较不同方法的优缺点
- 在中文文本上实践各种嵌入技术

In [ ]:
# 导入必要的库
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import jieba
import json

from src.text_vectorizer import TextVectorizer
from src.utils import load_documents, plot_vector_2d

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False   # 用来正常显示负号

## 1. 加载示例数据

首先让我们加载一些中文文档来进行实验。

In [ ]:
# 加载示例文档
documents = load_documents('../data/sample_documents.json')

print(f"加载了 {len(documents)} 个文档")
print("\n前3个文档:")
for i, doc in enumerate(documents[:3]):
    print(f"文档 {i+1}: {doc['title']}")
    print(f"内容: {doc['content'][:100]}...")
    print(f"类别: {doc['category']}")
    print("-" * 50)

## 2. TF-IDF向量化

TF-IDF (Term Frequency-Inverse Document Frequency) 是最经典的文本向量化方法之一。

### 原理
- **TF (词频)**: 词在文档中出现的频率
- **IDF (逆文档频率)**: 衡量词的重要性，常见词的IDF值较低
- **TF-IDF = TF × IDF**

In [ ]:
# 初始化文本向量化器
vectorizer = TextVectorizer()

# 提取文档内容
texts = [doc['content'] for doc in documents]

# 使用TF-IDF向量化
print("正在进行TF-IDF向量化...")
tfidf_vectors = vectorizer.tfidf_vectorize(texts)

print(f"TF-IDF向量维度: {tfidf_vectors.shape}")
print(f"向量稀疏度: {(tfidf_vectors == 0).sum() / tfidf_vectors.size * 100:.2f}%")

In [ ]:
# 可视化TF-IDF向量的特征分布
plt.figure(figsize=(12, 5))

# 每个文档的向量长度分布
plt.subplot(1, 2, 1)
vector_norms = np.linalg.norm(tfidf_vectors, axis=1)
plt.hist(vector_norms, bins=15, alpha=0.7, color='skyblue')
plt.title('TF-IDF向量长度分布')
plt.xlabel('向量长度')
plt.ylabel('文档数量')

# 特征值分布
plt.subplot(1, 2, 2)
non_zero_values = tfidf_vectors[tfidf_vectors > 0]
plt.hist(non_zero_values, bins=30, alpha=0.7, color='lightcoral')
plt.title('TF-IDF值分布(非零值)')
plt.xlabel('TF-IDF值')
plt.ylabel('频次')

plt.tight_layout()
plt.show()

## 3. Word2Vec嵌入

Word2Vec是一种将词映射到稠密向量空间的神经网络方法，能够捕获词之间的语义关系。

In [ ]:
# Word2Vec向量化
print("正在训练Word2Vec模型...")
word2vec_vectors = vectorizer.word2vec_vectorize(texts, vector_size=100, window=5, min_count=1)

print(f"Word2Vec向量维度: {word2vec_vectors.shape}")
print(f"向量稠密度: {(word2vec_vectors != 0).sum() / word2vec_vectors.size * 100:.2f}%")

In [ ]:
# 展示Word2Vec词向量的语义相似性
if hasattr(vectorizer, 'word2vec_model') and vectorizer.word2vec_model:
    model = vectorizer.word2vec_model
    
    # 找一些测试词
    test_words = ['技术', '人工智能', '体育', '艺术', '音乐']
    available_words = [word for word in test_words if word in model.wv.key_to_index]
    
    if available_words:
        print("词语相似性测试:")
        for word in available_words[:3]:
            try:
                similar_words = model.wv.most_similar(word, topn=3)
                print(f"与'{word}'最相似的词:")
                for similar_word, similarity in similar_words:
                    print(f"  {similar_word}: {similarity:.3f}")
                print()
            except KeyError:
                continue

## 4. Sentence Transformers嵌入

Sentence Transformers基于预训练的BERT等模型，能够生成高质量的句子级别嵌入。

In [ ]:
# Sentence Transformers向量化
print("正在使用Sentence Transformers进行向量化...")
try:
    sentence_vectors = vectorizer.sentence_transformer_vectorize(texts, model_name='all-MiniLM-L6-v2')
    print(f"Sentence Transformers向量维度: {sentence_vectors.shape}")
    print(f"向量稠密度: {(sentence_vectors != 0).sum() / sentence_vectors.size * 100:.2f}%")
except Exception as e:
    print(f"Sentence Transformers加载失败: {e}")
    print("将使用模拟向量进行演示")
    sentence_vectors = np.random.randn(len(texts), 384)  # 模拟384维向量

## 5. 向量化方法比较

让我们比较三种方法生成的向量的特性。

In [ ]:
# 比较不同方法的向量特性
methods = {
    'TF-IDF': tfidf_vectors,
    'Word2Vec': word2vec_vectors,
    'Sentence Transformers': sentence_vectors
}

plt.figure(figsize=(15, 5))

for i, (method_name, vectors) in enumerate(methods.items()):
    plt.subplot(1, 3, i+1)
    
    # 计算向量长度
    norms = np.linalg.norm(vectors, axis=1)
    plt.hist(norms, bins=15, alpha=0.7, label=method_name)
    plt.title(f'{method_name}\n向量长度分布')
    plt.xlabel('向量长度')
    plt.ylabel('文档数量')
    
    # 添加统计信息
    plt.axvline(np.mean(norms), color='red', linestyle='--', 
                label=f'均值: {np.mean(norms):.3f}')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 使用PCA降维可视化不同方法的向量分布
plt.figure(figsize=(15, 5))

categories = [doc['category'] for doc in documents]
unique_categories = list(set(categories))
colors = plt.cm.Set3(np.linspace(0, 1, len(unique_categories)))
category_color_map = dict(zip(unique_categories, colors))

for i, (method_name, vectors) in enumerate(methods.items()):
    plt.subplot(1, 3, i+1)
    
    # PCA降维到2D
    if vectors.shape[1] > 2:
        pca = PCA(n_components=2)
        vectors_2d = pca.fit_transform(vectors)
    else:
        vectors_2d = vectors
    
    # 按类别绘制散点图
    for category in unique_categories:
        mask = np.array(categories) == category
        plt.scatter(vectors_2d[mask, 0], vectors_2d[mask, 1], 
                   c=[category_color_map[category]], 
                   label=category, alpha=0.7, s=50)
    
    plt.title(f'{method_name}\nPCA可视化')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    if i == 0:  # 只在第一个图上显示图例
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

## 6. 相似度计算实验

让我们测试不同向量化方法在文档相似度计算上的表现。

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 选择一个查询文档
query_idx = 0
query_doc = documents[query_idx]

print(f"查询文档: {query_doc['title']}")
print(f"内容: {query_doc['content'][:100]}...")
print(f"类别: {query_doc['category']}")
print("\n" + "="*60)

# 对每种方法计算相似度
for method_name, vectors in methods.items():
    print(f"\n{method_name} 相似度排序:")
    
    # 计算查询向量与所有文档的相似度
    query_vector = vectors[query_idx].reshape(1, -1)
    similarities = cosine_similarity(query_vector, vectors)[0]
    
    # 排序（除了查询文档本身）
    similar_indices = np.argsort(similarities)[::-1][1:6]  # 前5个最相似的
    
    for i, idx in enumerate(similar_indices):
        sim_score = similarities[idx]
        doc = documents[idx]
        print(f"  {i+1}. {doc['title']} (相似度: {sim_score:.3f}) [{doc['category']}]")

## 7. 嵌入质量评估

评估不同向量化方法的质量，通过类内相似度和类间相似度来衡量。

In [ ]:
def evaluate_embedding_quality(vectors, categories):
    """评估嵌入质量"""
    similarities = cosine_similarity(vectors)
    
    # 计算类内和类间相似度
    intra_class_sims = []
    inter_class_sims = []
    
    for i in range(len(categories)):
        for j in range(i+1, len(categories)):
            sim = similarities[i, j]
            if categories[i] == categories[j]:
                intra_class_sims.append(sim)
            else:
                inter_class_sims.append(sim)
    
    return {
        'intra_class_mean': np.mean(intra_class_sims) if intra_class_sims else 0,
        'inter_class_mean': np.mean(inter_class_sims) if inter_class_sims else 0,
        'separation': np.mean(intra_class_sims) - np.mean(inter_class_sims) if intra_class_sims and inter_class_sims else 0
    }

# 评估所有方法
print("嵌入质量评估 (类内相似度 vs 类间相似度):")
print("="*60)

for method_name, vectors in methods.items():
    quality = evaluate_embedding_quality(vectors, categories)
    print(f"\n{method_name}:")
    print(f"  类内平均相似度: {quality['intra_class_mean']:.3f}")
    print(f"  类间平均相似度: {quality['inter_class_mean']:.3f}")
    print(f"  分离度 (越大越好): {quality['separation']:.3f}")

## 8. 实际应用场景

让我们演示如何在实际场景中选择合适的向量化方法。

In [ ]:
# 场景1: 关键词搜索 - TF-IDF通常表现良好
print("场景1: 关键词搜索")
print("查询: '人工智能技术发展'")

query_text = "人工智能技术发展"
query_tfidf = vectorizer.tfidf_vectorize([query_text])

# 计算与所有文档的相似度
similarities = cosine_similarity(query_tfidf, tfidf_vectors)[0]
top_indices = np.argsort(similarities)[::-1][:3]

print("\nTF-IDF搜索结果:")
for i, idx in enumerate(top_indices):
    print(f"  {i+1}. {documents[idx]['title']} (相似度: {similarities[idx]:.3f})")

In [ ]:
# 场景2: 语义搜索 - Sentence Transformers通常更好
print("\n场景2: 语义搜索")
print("查询: '创新科技应用'")

try:
    query_semantic = vectorizer.sentence_transformer_vectorize([query_text])
    similarities_semantic = cosine_similarity(query_semantic, sentence_vectors)[0]
    top_indices_semantic = np.argsort(similarities_semantic)[::-1][:3]
    
    print("\nSentence Transformers搜索结果:")
    for i, idx in enumerate(top_indices_semantic):
        print(f"  {i+1}. {documents[idx]['title']} (相似度: {similarities_semantic[idx]:.3f})")
except:
    print("Sentence Transformers不可用，跳过语义搜索演示")

## 9. 总结与建议

### 方法选择指南:

1. **TF-IDF**:
   - 优点: 简单快速，适合关键词匹配
   - 缺点: 无法捕获语义关系
   - 适用: 传统信息检索，关键词搜索

2. **Word2Vec**:
   - 优点: 捕获词语语义关系
   - 缺点: 需要聚合词向量到文档级别
   - 适用: 词语相似度分析，语义理解

3. **Sentence Transformers**:
   - 优点: 直接生成句子/文档级别嵌入，语义理解能力强
   - 缺点: 计算资源需求大，模型较大
   - 适用: 语义搜索，文档相似度，推荐系统

In [ ]:
# 保存实验结果
experiment_results = {
    'document_count': len(documents),
    'vector_dimensions': {
        'tfidf': tfidf_vectors.shape[1],
        'word2vec': word2vec_vectors.shape[1],
        'sentence_transformers': sentence_vectors.shape[1]
    },
    'quality_metrics': {}
}

for method_name, vectors in methods.items():
    quality = evaluate_embedding_quality(vectors, categories)
    experiment_results['quality_metrics'][method_name] = quality

print("实验完成！")
print(f"处理了{len(documents)}个文档")
print("各方法向量维度:", experiment_results['vector_dimensions'])